# Domande di Python, esempi di modelli in Pyomo

## 1. Domande ricevute
### Domanda 1: Il problema di *aliasing* nelle liste
Le liste in Python sono delle strutture dati *mutable*, ovvero che possono essere modificate, a differenze delle tuple che sono una struttura dati *read only*, ovvero che una volta che sono inizializzate non possono essere più modificate.

Quando si usano le liste, insieme all'operatore di assegnamento `=` (da non confondere con l'operatore logico di confronto `==`), quello che succedde è che non viene copiata la lista in una nuova variabile, ma viene creato un *handle* (un puntatore) alla stessa lista. Vediamo un esempio concreto di seguito.

In [ ]:
As = [1, 2, 3, 4]
Bs = As

Bs.pop(2)

print("As:", As)
print("Bs:", Bs)

Si noti che a tutti gli effetti, `Bs` reindrezza l'operazione `pop` alla struttura dati a cui si riferesce `As`. Usando una notazione **box-and-pointer**, quello che succede dovrebbe essere subito chiaro.

Per maggiori dettagli, si veda il paragrafo 4.4.1 del notebook seguente:
[Link al corso di Python](http://www-dimat.unipv.it/gualandi/programming/notebooks/Lab11_TupleListeDizionari.html)

Se uno avesse veramente voluto creare una copia della lista `As` in `Bs`, avrebbe potuto usare il costruttore delle list, o semplicemente un operatore di slicing, come nell'esempio seguente.

In [ ]:
As = [1, 2, 3, 4, 5]
Bs = list(As)
Cs = Bs[:]

Bs.pop(2)
Cs.pop(3)

print("As:", As)
print("Bs:", Bs)
print("Cs:", Cs)

### Domanda 2: Differenza tra modello e programma
Domanda ricevuta:

> [...] l'idea che mi sono fatto è che, in questo caso, sia sbagliato il modello di partenza, l'idea con cui intendo risolvere il problema. Ma poi mi chiedo: siccome non è la prima volta che riscontro un problema di questo tipo con pyomo, io non posso in alcun modo definire nuove variabili a partire dal valore che altre variabili assumono? 

La difficoltà principale nell'esercitazione sul CVRP è riuscire a mettere a fuoco la differenza tra l'implementare un **algoritmo** che risolve un problema, e invece tra troverro un **modello di programmazione lineare intera** che formalizza un problema usando solo

1. Variabili decisionali continue o intere
2. Funzione obiettivo lineare
3. Vincoli lineari

In particolare, sulla parte di domanda:

> non posso in alcun modo definire nuove variabili a partire dal valore che altre variabili assumono? 

La risposta è *yes, you can!*, ma devi riuscire a scrivere la relazione che vuoi esprimere tra le variabili in termini di vincoli lineari. Non devi implementare un algoritmo che calcola quella relazione.

Una volta scritto il modello, sarà un risolutore generico di programmazione lineare intera ad eseguire i suoi algoritmi, per cercare di risolvere il problema, e dichiarare il problema come (e dovete controllare lo status):

1. Non ammissibile
2. Illimitato
3. Risolto all'ottimo. 
4. Oppure: *raggiunto un time limit, o un raggiunto iteration limit, o esaurita le memoria...)

### Domanda 3: Variabili, parametri, assegnamento iniziale
Consideriamo il codice seguente:

In [ ]:
# Dati letti dal file:
Xs = [(266, 235), (295, 272), (301, 258), (309, 260), (217, 274), (218, 278), (282, 267), (242, 249), (230, 262), (249, 268), (256, 267), (265, 257), (267, 242), (259, 265), (315, 233), (329, 252), (318, 252), (329, 224), (267, 213), (275, 192), (303, 201), (208, 217), (326, 181)]
Ws = [0, 125, 84, 60, 500, 300, 175, 350, 150, 1100, 4100, 225, 300, 250, 500, 150, 100, 250, 120, 600, 500, 175, 75]

# Funzione distanza
from math import sqrt
def Distance(A, B):
    return sqrt((A[0]-B[0])**2 + (A[1] - B[1])**2)

In [ ]:
from pyomo.environ import ConcreteModel, Var, Objective, Constraint, SolverFactory, ConstraintList
from pyomo.environ import Binary, RangeSet, NonNegativeReals, NonNegativeIntegers, Reals, Maximize

model = ConcreteModel()

model.I = RangeSet(len(Xs))
model.J = RangeSet(len(Ws))
  
model.x = Var(model.I, model.J, within=Binary)
model.c = Var(model.I, model.J, within=NonNegativeReals)
for i in model.I:
    for j in model.J:
        if j is not i:
            model.x[i,j]()==0
            model.c[i,j]()==Distance(Xs[i],Xs[j]) 

## 2. Modelli in Pyomo
Di seguito vediamo come sia possibile scrivere alcuni modelli visti a lezione in Pyomo.

### 2.1 Problema di programmazione lineare casuale
Consideriamo la funzione seguente che crea un problema di Programmazione Lineare Intera (PLI) casuale.

Un problema di PLI è definito dai due vettori $c$ e $b$ e dalla matrice $A$:

$$
    z = \min \,\{ c x \mid Ax \geq b, x \geq 0 \,\}
$$

Scriviamo prima una funzione per generare i dati in modo casuale.

In [ ]:
from numpy import random
def RandomILP(n, m, seed=13):
    random.seed(seed)
    c = random.randint(1, 10, size=n)
    b = random.random(m)
    A = random.randint(1, 10, size=(m,n))
    
    return c, b, A
    
print(RandomILP(2,3))

Scriviamo ora una funzione che presi in input i dati di un'istanza, costruisce il modello, lo risolve, controlla lo status, e restituisce un eventuale soluzione.

In [ ]:
def SolveILP(c, b, A):
    m = len(b)
    n = len(c)
    
    model = ConcreteModel()

    model.I = RangeSet(n)
    model.J = RangeSet(m)
  
    model.x = Var(model.I, within=NonNegativeIntegers)
    
    model.obj = Objective(expr = sum(c[i-1]*model.x[i] for i in model.I))
    
    model.vincoli = ConstraintList()
    for j in model.J:
        model.vincoli.add(expr = sum(A[j-1,i-1]*model.x[i] for i in model.I) >= b[j-1])
        
    sol = SolverFactory('glpk').solve(model, tee=False)
    
    for info in sol['Solver']:
        print(info)
        
    xbar = [model.x[i]() for i in model.I]
    
    return 'optimal', model.obj(), xbar


In [ ]:
c, b, A = RandomILP(2, 3, 1717)
print(SolveILP(c, b, A))

### 2.2 Turnistica infermieri
Vediamo ora come possiamo scrivere il modello che rappresenta i turni settimanali degli infermieri del San Matteo di Pavia.

Per la descrizione del modello si vedano le slides del corso.


In [ ]:
def StaffScheduling():
    # Data
    d = 7
    w = 5
    demand = [5, 6, 7, 3, 3, 2, 2]
    
    # Dati sui turni
    A = np.zeros((7,7))
    for i in range(7):
        for j in range(5):
            A[(j+i)%7, i] = 1
            
    # Model
    model = ConcreteModel()

    model.Day = RangeSet(d)
    
    model.x = Var(model.Day, within=NonNegativeIntegers)
    
    model.obj = Objective(expr = sum(model.x[i] for i in model.Day))
    
    model.cover = ConstraintList()
    for day in model.Day:
        model.cover.add(expr = sum(A[day-1, i-1]*model.x[i] for i in model.Day) >= demand[day-1])
        
    model.write("staff.lp")
    
    sol = SolverFactory('glpk').solve(model, tee=True)
    
    for info in sol['Solver']:
        print(info)
        
    xbar = [model.x[i]() for i in model.Day]
    
    return 'optimal', model.obj(), xbar
    

In [ ]:
print(StaffScheduling())

In [ ]:
A = np.zeros((7,7))
for i in range(7):
    for j in range(5):
        A[(j+i)%7, i] = 1
        
A

### 2.3 RNA folding
Si consideri il modello visto a lezione nelle slides `RNA folding`, vediamo come implementare quel modello in Pyomo.

In [ ]:
from pyomo.environ import maximize

def RNAfold(s):
    C1 = ('A', 'U')
    C2 = ('C', 'G')
    
    ns = len(s)
    
    E = []
    for i in range(ns):
        for j in range(ns):
            if i < j:
                E.append( (i,j) )
                
    n = len(E)
    
    # Model
    model = ConcreteModel()
    
    model.P = RangeSet(n)
    
    model.x = Var(model.P, within=Binary)
    
    model.obj = Objective(expr = sum(model.x[p] for p in model.P),
                          sense = maximize)
    
    # Set variable to zero
    model.setzero = ConstraintList()
    for p in model.P:
        i,j = E[p-1]
        if not ((s[i],s[j]) == C1 or (s[j],s[i]) == C1 
                or (s[i],s[j]) == C2 or (s[j],s[i]) == C1):
            model.setzero.add(expr = model.x[p] == 0)
            
    # At most a single pair
    model.atmost = ConstraintList()
    for j in range(ns):
        L1 = []
        L2 = []
        for p in model.P:
            v,w = E[p-1]
            if j == w:
                L1.append( p )
            if j == v:
                L2.append( p )
        Ls = L1 + L2
        model.atmost.add(expr = sum(model.x[p] for p in Ls) <= 1)
        
    # Conflicts
    model.conf = ConstraintList()
    for p in model.P:
        i, j = E[p-1]
        for q in model.P:
            v, w = E[q-1]
            if i < v < j < w:
                model.conf.add(expr = model.x[p] + model.x[q] <= 1)
                
    sol = SolverFactory('glpk').solve(model, tee=True)
    
    for info in sol['Solver']:
        print(info)
        
    xbar = [model.x[i]() for i in model.P]
    
    return 'optimal', model.obj(), xbar


In [ ]:
s = 'ACGUGCCCGAU'
print(RNAfold(s))